# Geöffnete Skilifte absaugen



In [49]:
# Libraries laden
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Teil 1: Liste mit allen Live-Skigebieten finden

In [50]:
startseite = requests.get("https://www.bergfex.ch/schweiz/").text
soup_startseite = BeautifulSoup(startseite, "html.parser")

Die Links heraussuchen

In [51]:
liveskigebiete = soup_startseite.find_all('li', {'style':'font-weight: bold;'})
linkliste = []

for gebiet in liveskigebiete:
    link = gebiet.find('a')['href']
    linkliste.append(link)

In [52]:
pd.DataFrame(linkliste).to_csv('skigebiete_linkliste.csv')

## Teil 2: Öffnungszeiten auslesen
Leere Liste starten

In [53]:
skigebiete_final = []

Seiten absaugen & souppieren

In [61]:
for gebiet_link in linkliste:
    website = requests.get("https://www.bergfex.ch"+gebiet_link).text
    soup = BeautifulSoup(website, "html.parser")

    # Titel des Skigebiets
    titel = soup.find('h1', {"class": "has-sup"})
    titel = (titel.text.replace("Skigebiet ", ""))
    # Offene Skilifte auslesen
    for skigebiet in soup.find_all('table', class_="schneewerte"):
        try:
            offenelifte = (skigebiet.find_all('td')[-1].text).split(" von ") # Auslesen im Format "12 von 24" (offene Lifte sind immer das letzte TD, deshalb -1)
            total = offenelifte[1].strip()
            offen = offenelifte[0].strip()
            skigebiete_final.append({"titel": titel, "totallifte": total, "offen": offen}) # In die finale Liste laden
        except:
            print("Fehler mit Skigebiet: "+"https://www.bergfex.ch"+gebiet_link)



## Ergänzen in einer Tabelle

In [62]:
pd.DataFrame(skigebiete_final)

,offen,titel,totallifte
0,9,Arosa Lenzerheide,43
1,3,Corvatsch - Furtschellas,14
2,2,Davos Jakobshorn,10
3,5,Davos Klosters Parsenn,18
4,3,Flims Laax Falera,28
5,5,St. Moritz - Corviglia,23
6,1,Anzère,12
7,0,Champéry / Portes du Soleil,197
8,1,Grächen,9
9,0,Morgins / Portes du Soleil,197


## Teil 3: Alarmieren, wenn irgendetwas fehlschlägt

In [ ]:
# Send E-Mail